In [3]:
import numpy as np
import multiprocess
from p2_1 import get_image_feature
from p1 import compute_dictionary

def build_recognition_system(num_workers = 2):
    '''
    Creates a trained recognition system by generating training features from all training images.

    [input]
    * num_workers: number of workers to process in parallel

    [saved]
    * features: numpy.ndarray of shape (N,M)
    * labels: numpy.ndarray of shape (N)
    * dictionary: numpy.ndarray of shape (K,3F)
    * SPM_layer_num: number of spatial pyramid layers
    '''

    # Define the number of layers
    spm_layers = 7
    num_clusters = 250 # K in k means
    alpha = 200

    # Load the dictionary and training data
    train_data = np.load("./data/train_data.npz")
    compute_dictionary(alpha=alpha, num_clusters=num_clusters, out_name="dict_tuning")
    dictionary = np.load("dict_tuning.npy")

    # Parse out the training files
    train_files = train_data.get("files")
    if train_files is None:
        raise ValueError("No valid training files available :(")
    train_files = ["./data/" + str(file) for file in train_files]

    # Parse out the training labels
    labels = train_data.get("labels")
    if labels is None:
        raise ValueError("No valid labels available :(")

    # Gather arguments for multiprocessing
    args = []
    for idx, train_sample in enumerate(train_files):
        args.append((train_sample, dictionary, spm_layers, dictionary.shape[0]))

    # NxM array of the training features
    training_features = np.zeros((len(args), int(dictionary.shape[0] / 3 * (4**(spm_layers)-1))))

    # Multiprocess getting the image features
    result_dict = {}
    with multiprocess.Pool(num_workers) as p:
        # Get the result
        result = p.starmap(get_image_feature, args)

        for r in result:
            # Associate where to put it in the output list
            idx = np.argwhere(np.array(train_files) == r[0])

            # Put it in the output list
            training_features[idx[0]] = r[1]

    np.savez('trained_system_edited.npz', features=training_features,
                                    labels=labels,
                                    dictionary=dictionary,
                                    SPM_layer_num=spm_layers)


build_recognition_system()

TypeError: compute_dictionary() got an unexpected keyword argument 'alpha'